# Please find below the high-level structuring of python scripts in the attached folder. 

## Feature Engineering Script: 
* Feature Engineering_PPM  --- PPM, PRNR, Premium Freight

## Individual Model Scripts:
* regression approach PPM-v3
* regression approach prem freight 
* regression approach prnr
* Vulnerability Score Reg ----contains the feature engineering piece of vulnerability score as well

## Prod Scripts:
* Prod Script_V1   --- PPM, PRNR, Premium Freight
* Prod Script Vulnerability Score  ----- Vulnerability Score

## We need to run the prod scripts in the 1st week of every month

In [2]:
import pandas as pd
import numpy as np
import re
import os
from dateutil.relativedelta import relativedelta
import cx_Oracle
os.chdir(r"C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\Supplier_Risk")

### Import Data Files

In [2]:
tns_dsn = cx_Oracle.makedsn('aamlxbidbp001.aam.net', 1525, service_name='BIDWPRD')
con_ORACLE = cx_Oracle.connect(user='PROD_AAM_RO', password='prodaamro', dsn=tns_dsn)

df = pd.read_sql_query("""select * from bi_dw.WC_SC_SUMMARY_V
                       where month >= 201801
                       """, con_ORACLE)

In [3]:
df.to_csv(os.getcwd()+r"\data\download_backup11.csv", index= False)


In [4]:
# df.to_csv(os.getcwd()+r"\data\download_backup.csv", index= False)

df = pd.read_csv(os.getcwd()+r"\data\download_backup.csv")

In [5]:
df.head()

,TYPE,VENDOR_NAME,ORGANIZATION_CODE,MONTH,VAL
0,Qty Recd,*AAM INTERNATIONAL S.A.R.L. (LUXCO METALFORM),LDD,201812,227751.0
1,Qty Recd,*AAM INTERNATIONAL S.A.R.L. (LUXCO METALFORM),LDD,201904,142328.0
2,Qty Recd,*CHEVRON BRASIL LUBRIFICANTES LTDA,ABD,201807,56000.0
3,Qty Recd,*CHEVRON BRASIL LUBRIFICANTES LTDA,ABD,201901,31000.0
4,Qty Recd,*CHEVRON BRASIL LUBRIFICANTES LTDA,ABD,201902,17000.0


In [6]:
df.tail()

,TYPE,VENDOR_NAME,ORGANIZATION_CODE,MONTH,VAL
262595,PPM Qty Rejected,FORTECH PRODUCTS INC,ABD,202210,0.0
262596,PPM Qty Rejected,GTS ENG CO LTD,PYD,202208,0.0
262597,PPM Qty Rejected,AMERICAN AXLE & MANUFACTURING INC,AFT,202208,0.0
262598,PPM Qty Rejected,WEDGE PRODUCTS INC,AFT,202208,0.0
262599,PPM Qty Rejected,CENVISION TECHNOCRAT,PCD,202208,0.0


In [7]:
# df = df[df['MONTH'] != 202209]
# df = df[df['MONTH'] != 202210]
# df

In [8]:
df.TYPE.value_counts()

Qty Recd            113388
PPM Qty Rejected     50971
PPM Qty Shipped      50971
Qty Ordered          37872
PRNR                  5503
Cost Recovery         3538
PREM FREIGHT           357
Name: TYPE, dtype: int64

In [9]:
# Download data from Orcale DB for prod
# Logic to collate data and filter suppliers to be implemented in the SQL script itself
# df = pd.read_csv(os.getcwd()+r"\Data Files\Feature Engineering Base.csv", encoding='latin-1')
df1 = pd.read_csv(os.getcwd()+r"\data\Mapping File Prod.csv", encoding='latin-1')

In [10]:
df = df.merge(df1, on = ["VENDOR_NAME","ORGANIZATION_CODE"], how='left')

In [11]:
# df = df[df["ORG_TYPE"]!="Others2"]
df = df.groupby(['VENDOR_NAME', 'MONTH', 'TYPE']).sum().reset_index()

In [12]:
df.head()

,VENDOR_NAME,MONTH,TYPE,VAL
0,*4FASTENING GMBH,202109,Qty Recd,1600.0
1,*A.M.S. COMERCIO DE SENSORES DO PARANA LTDA.,201903,Qty Recd,11.0
2,*AAM DE MEXICO SA DE CV (FORGE DIVISION),201801,Qty Recd,800.0
3,*AAM INTERNATIONAL S.A.R.L. (ACCUGEAR SILAO),201802,Qty Recd,10.0
4,*AAM INTERNATIONAL S.A.R.L. (ACCUGEAR SILAO),201803,Qty Recd,120.0


In [13]:
df = df.pivot(index = ['VENDOR_NAME','MONTH'], columns=['TYPE'], values = 'VAL').reset_index()

In [14]:
df.head()

TYPE,VENDOR_NAME,MONTH,Cost Recovery,PPM Qty Rejected,PPM Qty Shipped,PREM FREIGHT,PRNR,Qty Ordered,Qty Recd
0,*4FASTENING GMBH,202109,NaN,NaN,NaN,NaN,NaN,NaN,1600.0
1,*A.M.S. COMERCIO DE SENSORES DO PARANA LTDA.,201903,NaN,NaN,NaN,NaN,NaN,NaN,11.0
2,*AAM DE MEXICO SA DE CV (FORGE DIVISION),201801,NaN,NaN,NaN,NaN,NaN,NaN,800.0
3,*AAM INTERNATIONAL S.A.R.L. (ACCUGEAR SILAO),201802,NaN,NaN,NaN,NaN,NaN,NaN,10.0
4,*AAM INTERNATIONAL S.A.R.L. (ACCUGEAR SILAO),201803,NaN,NaN,NaN,NaN,NaN,NaN,120.0


### Data Cleaning 

In [15]:
df["Qty Recd"].fillna(0, inplace = True)

In [16]:
df["QTY_RECD_FLAG"] = np.where((df["Qty Ordered"] != np.NaN) & (df["Qty Ordered"] !=0), df["Qty Recd"]/df["Qty Ordered"], np.NaN)
df["QTY_RECD_FLAG"] = np.where(df["QTY_RECD_FLAG"]>1, 1, df["QTY_RECD_FLAG"])

In [17]:
df["PPM Flag"] = np.where((df["PPM Qty Shipped"] != np.NaN) & (df["PPM Qty Shipped"] != 0), df["PPM Qty Rejected"]/df["PPM Qty Shipped"]*1000000, np.NaN)

In [18]:
df["MONTH"] = pd.to_datetime(df["MONTH"], format = '%Y%m').dt.date

In [19]:
target_df = df[["VENDOR_NAME"]]
target_df.drop_duplicates(inplace = True)

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### QTY_RECD_FLAG to be aggregated w/o creating flag earlier

In [4]:
from datetime import date, datetime

In [5]:
for month in [datetime(2022,x,1).date() for x in np.arange(1,3)] +[datetime(2021,x,1).date() for x in np.arange(1,12)]+[datetime(2020,x,1).date() for x in np.arange(1,12)]+[datetime(2019,x,1).date() for x in np.arange(1,12)]:
    print(month)

2022-01-01
2022-02-01
2021-01-01
2021-02-01
2021-03-01
2021-04-01
2021-05-01
2021-06-01
2021-07-01
2021-08-01
2021-09-01
2021-10-01
2021-11-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01


In [20]:
final_target_df = pd.DataFrame()


for month in [pd.datetime(2022,x,1).date() for x in np.arange(1,3)] +[pd.datetime(2021,x,1).date() for x in np.arange(1,12)]+[pd.datetime(2020,x,1).date() for x in np.arange(1,12)]+[pd.datetime(2019,x,1).date() for x in np.arange(1,12)]:
#     month = pd.datetime(2021,12,1)
    target_df = df[df["MONTH"]==month][["VENDOR_NAME","MONTH","PRNR","PREM FREIGHT", "PPM Flag"]]
    target_df.drop_duplicates(inplace = True)


    # Month-on-Month features
    for z in np.arange(1,13,1):
        tmp = df[df["MONTH"]==month - relativedelta(months=z)].groupby(['VENDOR_NAME']).agg({ 
                    "QTY_RECD_FLAG" : ['mean'],
                    "Qty Ordered":['sum'], 
                    "Qty Recd":['sum'],
                    "PRNR" : ['sum'],
                    "PREM FREIGHT" : ['sum'],
                    "Cost Recovery" : ['sum'],
                    "PPM Qty Rejected": ['sum'],
                    "PPM Qty Shipped":['sum']
                    })
        tmp.columns = [s + "_MOM_"  + str(z) for s in ["_".join(x) for x in tmp.columns.ravel()]]
        tmp.reset_index(inplace = True)
        target_df = target_df.merge(tmp, how = 'left', on = 'VENDOR_NAME')


    #Quarterly features
    for z in np.arange(1,13,3):
        tmp = df[(df["MONTH"]>=month - relativedelta(months=z+2))&(df["MONTH"]<=month - relativedelta(months=z))].groupby(['VENDOR_NAME']).agg({ 
                    "QTY_RECD_FLAG" : ['max','min','std','mean'],
                    "Qty Ordered":['max','min','std','mean', 'sum'], 
                    "Qty Recd":['max','min','std','mean','sum'],
                    "PRNR" : ['max','std','mean','sum'],
                    "PREM FREIGHT" : ['max','std','mean','sum'],
                    "Cost Recovery" : ['max','std','mean','sum'],
                    "PPM Flag" : ['max','min','std','mean'], 
                    "PPM Qty Rejected": ['max','min','std','mean', 'sum'],
                    "PPM Qty Shipped":['max','min','std','mean', 'sum']
                    })
        tmp.columns = [s + "_OQT_"  + str(z) for s in ["_".join(x) for x in tmp.columns.ravel()]]
        tmp.reset_index(inplace = True)
        for z in [x for x in tmp.columns if "PPM Qty Shipped_sum" in x]:
            tmp[z.replace("PPM Qty Shipped_sum","PPM_Agg_Flag")] = np.where((tmp[z]!= np.NaN) & (tmp[z]!= 0), np.round(tmp[z.replace("Shipped","Rejected")]/tmp[z]*1000000), np.NaN)
        
        for z in [x for x in tmp.columns if "Qty Ordered_sum" in x]:
            tmp[z.replace("Qty Ordered_sum","Qty Recd_percent_Agg")] = np.where((tmp[z]!= np.NaN) & (tmp[z]!= 0), tmp[z.replace("Ordered","Recd")]/tmp[z]*100, np.NaN) 

        target_df = target_df.merge(tmp, how = 'left', on = 'VENDOR_NAME')
        
        
    final_target_df = pd.concat([final_target_df, target_df])

C:\Users\Kumbhar\OneDrive - American Axle & Manufacturing, Inc\AAM\globle_env\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  after removing the cwd from sys.path.


In [21]:
final_target_df.head()

,VENDOR_NAME,MONTH,PRNR,PREM FREIGHT,PPM Flag,QTY_RECD_FLAG_mean_MOM_1,Qty Ordered_sum_MOM_1,Qty Recd_sum_MOM_1,PRNR_sum_MOM_1,PREM FREIGHT_sum_MOM_1,...,PPM Qty Rejected_std_OQT_10,PPM Qty Rejected_mean_OQT_10,PPM Qty Rejected_sum_OQT_10,PPM Qty Shipped_max_OQT_10,PPM Qty Shipped_min_OQT_10,PPM Qty Shipped_std_OQT_10,PPM Qty Shipped_mean_OQT_10,PPM Qty Shipped_sum_OQT_10,PPM_Agg_Flag_OQT_10,Qty Recd_percent_Agg_OQT_10
0,*CERATIZIT CHICAGO INC,2022-01-01,NaN,NaN,NaN,NaN,0.0,3.0,0.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
1,*KAMAX LP,2022-01-01,NaN,NaN,NaN,1.0,496800.0,574000.0,1.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,90.740741
2,2A MATERIAIS ELETRICOS LTDA,2022-01-01,NaN,NaN,NaN,NaN,0.0,2.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3 DIMENSIONAL SERVICES,2022-01-01,NaN,NaN,NaN,NaN,0.0,115.0,0.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4,A & F GROBSERIEN GMBH,2022-01-01,NaN,NaN,0.0,1.0,14364.0,28164.0,0.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,98.726223


In [27]:
final_df_col = list(final_target_df)

In [28]:
final_df_col

['VENDOR_NAME',
 'MONTH',
 'PRNR',
 'PREM FREIGHT',
 'PPM Flag',
 'QTY_RECD_FLAG_mean_MOM_1',
 'Qty Ordered_sum_MOM_1',
 'Qty Recd_sum_MOM_1',
 'PRNR_sum_MOM_1',
 'PREM FREIGHT_sum_MOM_1',
 'Cost Recovery_sum_MOM_1',
 'PPM Qty Rejected_sum_MOM_1',
 'PPM Qty Shipped_sum_MOM_1',
 'QTY_RECD_FLAG_mean_MOM_2',
 'Qty Ordered_sum_MOM_2',
 'Qty Recd_sum_MOM_2',
 'PRNR_sum_MOM_2',
 'PREM FREIGHT_sum_MOM_2',
 'Cost Recovery_sum_MOM_2',
 'PPM Qty Rejected_sum_MOM_2',
 'PPM Qty Shipped_sum_MOM_2',
 'QTY_RECD_FLAG_mean_MOM_3',
 'Qty Ordered_sum_MOM_3',
 'Qty Recd_sum_MOM_3',
 'PRNR_sum_MOM_3',
 'PREM FREIGHT_sum_MOM_3',
 'Cost Recovery_sum_MOM_3',
 'PPM Qty Rejected_sum_MOM_3',
 'PPM Qty Shipped_sum_MOM_3',
 'QTY_RECD_FLAG_mean_MOM_4',
 'Qty Ordered_sum_MOM_4',
 'Qty Recd_sum_MOM_4',
 'PRNR_sum_MOM_4',
 'PREM FREIGHT_sum_MOM_4',
 'Cost Recovery_sum_MOM_4',
 'PPM Qty Rejected_sum_MOM_4',
 'PPM Qty Shipped_sum_MOM_4',
 'QTY_RECD_FLAG_mean_MOM_5',
 'Qty Ordered_sum_MOM_5',
 'Qty Recd_sum_MOM_5',
 'PRN

In [ ]:
"MONTH","PRNR","PREM FREIGHT","PRNR_TARGET", "PREM_FREIGHT_TARGET

In [23]:
final_target_df.to_csv(os.getcwd()+r"\data\features_file.csv", index = False)